In [1]:
import numpy as np
import pandas as pd
import math
import os
import ciso8601
from datetime import datetime
from tqdm import tqdm

In [2]:
time_interval  = 29
time_interval_str = str(time_interval)
time_in_second = (time_interval - 1) * 3600

In [3]:
#用crea標的時間也要改
patientList = pd.read_csv('../processing_data/label1/' + time_interval_str + '/patient_list.csv')


In [4]:
crea = pd.read_csv('../processing_data/label2/' + time_interval_str + '/crea_csv.csv')
crea = crea.drop(columns=(crea.columns[0]), axis = 1)
crea = crea.rename(columns = {'0': 'Index','1': 'Method','2': 'Timestamp', '3' : 'Value'})
crea_time = []
for c in crea['Timestamp']:
    crea_time.append(ciso8601.parse_datetime(c).timestamp())
crea = crea.drop(columns = 'Timestamp')
crea['Time'] = crea_time
crea = crea.sort_values(by='Time')
crea = crea.reset_index(drop=True)
print(crea)

           Index Method Value          Time
0       15831921     血液  1.21  1.388605e+09
1       15831921     血液  1.21  1.388605e+09
2       15991230     血液  0.77  1.388646e+09
3       15991230     血液  0.77  1.388646e+09
4       10342202     血液  0.89  1.388652e+09
...          ...    ...   ...           ...
141944    272742     血液  0.97  1.614384e+09
141945    272742     血液  0.97  1.614384e+09
141946  11126820     血液  5.50  1.614384e+09
141947   2168102     血液  1.44  1.614398e+09
141948    831618     血液  3.13  1.614430e+09

[141949 rows x 4 columns]


AKI is defined as any of the following (Not Graded):
1. K Increase in SCr by >=0.3 mg/dl (>=26.5 lmol/l) within 48 hours; or
2. K Increase in SCr to >=1.5 times baseline, which is known or presumed to have occurred within the prior 7 days; or
3. K Urine volume <0.5 ml/kg/h for 6 hours

In [5]:
def kdigo_crea(time_pre, value_pre, time_now, value_now):
    if '>' in value_now or '>' in value_pre or '/' in value_now or '/' in value_pre:
        return False
    if '<' in value_pre:
        value_pre = 0.19
    if '<' in value_now:
        value_now = 0.19
    if (int(time_now) - int(time_pre) <= (3600 * 48)) and ((int(time_now) > int(time_pre)))  and (float(value_now) - float(value_pre) >= 0.3):
        return True
    elif (int(time_now) - int(time_pre) <= (3600 * 24 * 7)) and (int(time_now) > int(time_pre))  and (float(value_now)/float(value_pre) >= 1.5):         
        return True                                                         

In [6]:
########resolve copy problem
pd.options.mode.chained_assignment = None 
base_change = 0
label_change = 0
drop_list = []
for i in tqdm(range(len(patientList))):
    end_time = 0
    temp = crea['Index'].isin([patientList['patient_code'][i]])
    cur = temp[temp == True].index.tolist()
    
    if len(cur) >= 2:
        for j in range(len(cur)-1):
            if crea['Method'][cur[j]] == '尿液':
                continue
            elif crea['Time'][cur[j]] + 28800 < patientList['icu_time'][i] or  crea['Time'][cur[j]] + 28800 > patientList['leave_time'][i]:
                continue
            for k in range(j + 1, len(cur)):
                if crea['Method'][cur[k]] == '尿液':
                    continue
                elif crea['Time'][cur[k]] + 28800 < patientList['icu_time'][i] or  crea['Time'][cur[k]] + 28800 > patientList['leave_time'][i]:
                    continue
                if kdigo_crea(crea['Time'][cur[j]], crea['Value'][cur[j]], crea['Time'][cur[k]], crea['Value'][cur[k]]) == True:
                    end_time = int(crea['Time'][cur[k]])
                    break
            if end_time != 0:
                break
                
        if end_time != 0:
            if patientList['end_time'][i] > (end_time + 28800 - 3600) :
                if patientList['icu_time'][i] > end_time + 28800 - 3600 - time_in_second: #太早發病
                    # print(patientList['patient_code'][i])
                    drop_list.append(i)
                else:
                    patientList['end_time'][i]  = end_time + 28800 - 3600
                    patientList['start_time'][i] = end_time + 28800 - 3600 - time_in_second
                    if patientList['label'][i] == 0:
                        patientList['label'][i] = 1
                        label_change += 1
                    patientList['base'][i] = 'c'
                    base_change += 1
                
print('改成用crea標記: ' + str(base_change))
print('用crea判斷會發生aki: ' + str(label_change))
print('太早發病: ' + str(len(drop_list)))
                


100%|█████████████████████████████████████████████████████████████████████████████| 6578/6578 [00:35<00:00, 185.80it/s]

改成用crea標記: 327
用crea判斷會發生aki: 107
太早發病: 204


In [7]:
patientList.drop(drop_list, axis = 0, inplace = True)
patientList = patientList.reset_index(drop = True)
print(patientList.shape)

(6374, 9)


In [8]:
if not os.path.exists('../processing_data/label3/' + time_interval_str):
    os.mkdir('../processing_data/label3/' + time_interval_str)
patientList.to_csv('../processing_data/label3/' + time_interval_str + '/new_patient_list.csv', index = False, encoding = 'utf-8')